# 写作概览

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
import os
os.chdir('../..')
print(os.getcwd())

/Users/xuehongwei/github/langchain_chinese


## 构建写作链

In [4]:
from langchain_chinese import BookWritingChain
from langchain_zhipu import ChatZhipuAI

book = BookWritingChain(config_path="./notes/writing", output_dir="notes/writing", llm=ChatZhipuAI())
book_chain = book.get_chain(
    styles="列表时使用： 1) 2) 这样的序号", 
    demo="""
    1）全名：欧阳娜娜，小名：娜娜；
    2）全名：欧阳娜娜，小名：娜娜；    
    """
)

In [5]:
 book_chain.get_prompts()[0]

ChatPromptTemplate(input_variables=['input', 'knowledge'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': [], 'demo': '\n    1）全名：欧阳娜娜，小名：娜娜；\n    2）全名：欧阳娜娜，小名：娜娜；    \n    ', 'goal': '暂无。', 'instruction': '暂无。', 'rules': '暂无。', 'styles': '列表时使用： 1) 2) 这样的序号'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['demo', 'goal', 'instruction', 'knowledge', 'rules', 'styles'], template='\n你是强大的AI写作助手。\n\n## 写作目标\n{goal}\n\n## 指导原则\n{instruction}\n\n## 你的思考过程必须考虑下面提供的资料内容\n{knowledge}\n\n## 你的输出不得与如下观点相违背\n{rules}\n\n## 你的写作风格必须符合以下要求\n{styles}\n\n## 你的输出内容必须参考如下示例\n{demo}\n')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptT

In [6]:
prompt_value = book_chain.get_prompts()[0].invoke({"input": "「我的问题」", "knowledge": "「查询到的背景知识」"})
print(prompt_value.messages[0].content)


你是强大的AI写作助手。

## 写作目标
暂无。

## 指导原则
暂无。

## 你的思考过程必须考虑下面提供的资料内容
「查询到的背景知识」

## 你的输出不得与如下观点相违背
暂无。

## 你的写作风格必须符合以下要求
列表时使用： 1) 2) 这样的序号

## 你的输出内容必须参考如下示例

    1）全名：欧阳娜娜，小名：娜娜；
    2）全名：欧阳娜娜，小名：娜娜；    
    



In [7]:
book_chain.invoke("给我5个人名的列表")

-------------------- OUTPUT_PATH: /Users/xuehongwei/github/langchain_chinese/notes/writing/index.md --------------------
以下是五个人名的列表：

1）全名：李华，小名：华华；
2）全名：王丽，小名：丽丽；
3）全名：张伟，小名：伟伟；
4）全名：刘婷，小名：婷婷；
5）全名：陈敏，小名：敏敏。
--------------------


AIMessage(content='以下是五个人名的列表：\n\n1）全名：李华，小名：华华；\n2）全名：王丽，小名：丽丽；\n3）全名：张伟，小名：伟伟；\n4）全名：刘婷，小名：婷婷；\n5）全名：陈敏，小名：敏敏。')

In [3]:
for chunk in book_chain.stream("给我5个人名的列表"):
    print(chunk)

content='以下是' id='8577962834629537596'
content='五' id='8577962834629537596'
content='个人' id='8577962834629537596'
content='名的' id='8577962834629537596'
content='列表' id='8577962834629537596'
content='：\n\n1' id='8577962834629537596'
content='）' id='8577962834629537596'
content='全' id='8577962834629537596'
content='名' id='8577962834629537596'
content='：' id='8577962834629537596'
content='张' id='8577962834629537596'
content='三' id='8577962834629537596'
content='，' id='8577962834629537596'
content='小' id='8577962834629537596'
content='名' id='8577962834629537596'
content='：' id='8577962834629537596'
content='三' id='8577962834629537596'
content='儿' id='8577962834629537596'
content='；\n2' id='8577962834629537596'
content='）' id='8577962834629537596'
content='全' id='8577962834629537596'
content='名' id='8577962834629537596'
content='：' id='8577962834629537596'
content='李' id='8577962834629537596'
content='四' id='8577962834629537596'
content='，' id='8577962834629537596'
content='小' id='857796283

## 快速构建

In [39]:
translate_prompt_kv = {
  "goal": """
    如果给定的文字是中文，就请帮我为英文；如果给定的文字是其他语言，就帮我翻译为中文
    """,

  "instruction": """
    1. 请尽量保持文章的原来意思和原来的风格，不要额外发挥。
    2. 直接给出翻译结果即可。
    3. 语言尽量简练，不要啰嗦，不要增加补充提问。
    """,

  "demo": """
    ```
    user: hi
    assistant: 你好
    user: 你好
    assistant: hi 
    ```
    """
}

translate_chain = book.get_chain(**translate_prompt_kv)
translate_chain.invoke("听说广州鸿蒙公司有一个炳神")

AIMessage(content='It is reported that there is a figure named Bing Shen in the Guangzhou Hongmeng Company. \n\n(Note: The given information does not directly relate to the content provided for translation. If "炳神" is a term with specific connotations or is a person\'s nickname within the context of the Hongmeng company or related industries, a direct translation might not fully capture the meaning. Additional context would be required for a more accurate translation.)')

In [40]:
translate_chain.invoke("听说广州鸿蒙公司有一个叫炳神的家伙")

AIMessage(content='There is a guy named Bing Shen at Guangzhou Hongmeng Company.')

## 文案整体规划

In [41]:
translate_prompt_kv = {
  "goal": """
    我正在写一个较长的文案书，请你帮我列出所需的主要章节。
    """,

  "instruction": """
    1. 语言尽量简练，不要啰嗦，不要增加补充提问。
    """,

  "demo": """
    ```
XX系统升级方案建议20240415

（一）需求分析
一、旧系统升级
二、新增主动作为监督模块的需求

（二）工作内容
一、旧系统运维
二、XXXX模块
这部分工作职责涉及XXX，并建立YYYY的机制。目标在于一方面XXXX，另一方面XXXXX。
三、新基础平台设计
这部分工作涉及将旧系统的功能与新系统功能进行融合，目标是最终实现新系统的独立运作，并逐步淘汰旧系统。

（三）调研工作开展
一、数据调研工作
二、旧系统相关调研工作
三、XXXXX的接口调研工作开展 
    ```
    """
}

book_chain = book.get_chain(**translate_prompt_kv)
for chunk in book_chain.stream("重庆市市场监督新媒体监督系统升级改造建议"):
    print(chunk.content, end="")

```
重庆市市场监督新媒体监督系统升级改造建议

（一）项目背景与需求分析
一、现有系统状况概述
二、技术升级的必要性
三、新媒体监督功能扩展需求

（二）升级改造工作内容
一、现有系统维护与优化
   1. 系统漏洞修复
   2. 用户反馈收集与处理
二、新媒体监督模块构建
   1. 新媒体数据采集与分析
   2. 风险预警与应急响应机制设立
三、综合管理平台设计
   目标：实现多模块信息整合，提高监督效率。

（三）技术与市场调研
一、国内外新媒体监督系统发展调研
二、现有系统技术瓶颈与升级方案调研
三、用户需求与市场趋势调研

（四）系统升级实施方案
一、技术路线规划
二、人员培训与团队建设
三、升级时间表与里程碑

（五）风险评估与应对措施
一、技术风险评估
二、数据安全与隐私保护
三、应急预案制定

（六）预期效果与效益分析
一、系统功能提升
二、监督效能增强
三、社会效益与经济效益评估

（七）后期维护与服务体系建设
一、系统运行监控
二、用户服务支持
三、持续优化与升级策略

（八）总结与建议
一、项目总结
二、政策与资金支持建议
三、未来发展方向展望
```

以上章节设置旨在简练地概述从现有系统分析到后期维护的全过程，确保项目升级改造的全面性和可行性。

## 章节内小节规划

In [42]:
translate_prompt_kv = {
  "goal": """
    请帮我规划指定章节下应当包含哪些小节。请给出小节标题和摘要。
    """,

  "instruction": """
    1. 语言尽量简练，不要啰嗦，不要增加补充提问。
    """,

  "demo": """
    ```
    依据政府颁布的资金补助和扶持企业政策，个人与企业有资格获得支持。然而，在政策执行过程中，可能由于某些公职人员的不作为、慢作为或乱作为，导致出现不公平、不公正、不透明的处理结果，使得本不应享受优惠的群体得到了利益，而那些本应受益的个人或企业却被排除在外。
    纪委需针对这一风险采取行动，对相关部门的违规行为进行审查，对不积极、效率低下或不当行为进行深入调查和处理，迅速纠正错误行为，实施整改措施，追回不当利益，确保“应得”群体能够享受到政策红利，从而提升政策执行的实效。 
    ```
    """
}

book_chain = book.get_chain(**translate_prompt_kv)
for chunk in book_chain.stream("新媒体监督趋势分析"):
    print(chunk.content, end="")

根据您的要求，以下是针对“新媒体监督趋势分析”的章节规划，包含小节标题和摘要。

### 第一章：新媒体监督概述

#### 1.1 新媒体监督的发展历程
摘要：简述新媒体监督的起源，以及在我国的发展历程和各个阶段的特点。

#### 1.2 新媒体监督的现况
摘要：分析当前新媒体监督在舆论引导、社会监督等方面的作用及存在的问题。

### 第二章：新媒体监督的技术演进

#### 2.1 5G时代新媒体监督的技术革新
摘要：探讨5G技术对新媒体监督的影响，包括信息传播速度、数据处理能力等方面的提升。

#### 2.2 人工智能、区块链等技术在监督中的应用
摘要：分析人工智能、区块链等新技术在新媒体监督中的应用前景和挑战。

### 第三章：新媒体监督的案例分析

#### 3.1 典型案例回顾
摘要：回顾新媒体监督中的典型事件，分析其对社会监督和政策改革的推动作用。

#### 3.2 案例中存在的问题及反思
摘要：总结案例中反映出的问题，对新媒体监督的不足进行反思。

### 第四章：新媒体监督的发展趋势

#### 4.1 监督与伦理规范的建设
摘要：探讨在新媒体监督中加强伦理规范的重要性，以及如何构建健康的网络监督环境。

#### 4.2 新媒体与传统媒体的融合
摘要：分析新媒体与传统媒体在监督领域的融合趋势，以及如何优势互补，提高监督效果。

### 第五章：新媒体监督的政策与建议

#### 5.1 政策制定与落实
摘要：探讨政府在新媒体监督中的作用，以及如何制定和落实相关政策。

#### 5.2 对未来的展望与建议
摘要：从多方面对未来新媒体监督的发展提出展望，为提升监督效果提出具体建议。

以上规划旨在全面、简练地阐述新媒体监督的趋势，希望对您的写作有所帮助。

## 细节内容扩写

In [43]:
translate_prompt_kv = {
  "goal": """
    请帮对这一小节做详细的展开扩写。
    在扩写时，请对于涉及的主要概念，请专门段落解释，放置在内容最前。
    """,

  "instruction": """
    1. 语言尽量简练，不要啰嗦，不要增加补充提问。
    """,

  "demo": """
    ```
    ```
    """
}

book_chain = book.get_chain(**translate_prompt_kv)
for chunk in book_chain.stream(""""
### 第一章：新媒体监督的发展历程
#### 1.1 新媒体监督的起源
**摘要**：回顾新媒体监督的起始阶段，探讨其与传统媒体监督的区别及优势。
"""):
    print(chunk.content, end="")

#### 1.1.1 新媒体的崛起

新媒体，这个概念在21世纪初开始广泛流行，主要指依托于互联网技术，通过数字化手段传播信息的新兴媒体形态。它包括了社交媒体、网络新闻平台、博客、播客等多种形式。新媒体的崛起打破了传统媒体（如报纸、电视、广播）对信息传播的垄断，使信息传播更加快速、广泛和互动。

#### 1.1.2 新媒体监督的优势

新媒体监督相较于传统媒体监督，展现出明显的时代特征和优势。首先，新媒体的信息传播速度快，能够实时更新，及时反映社会事件和问题。其次，新媒体的互动性强，用户不仅可以接收信息，还可以发布信息、参与讨论，形成了一种全民参与的监督模式。此外，新媒体的传播范围广，不受地域限制，使得监督行为更加广泛和深入。

#### 1.1.3 新媒体监督与传统媒体监督的区别

新媒体监督与传统媒体监督最本质的区别在于传播介质和互动性。传统媒体往往依赖于特定的传播渠道，如报纸、电视等，而新媒体则通过互联网这个开放平台进行信息交流。在互动性方面，传统媒体多为单向传播，而新媒体的双向互动传播机制使得信息反馈更加及时，有助于形成更为有效的社会监督。

### 第二章：新媒体时代传统媒体的应对策略

#### 2.1 传统媒体面临的挑战

在新媒体时代，传统媒体面临了前所未有的挑战。信息的快速更迭和用户获取信息习惯的改变使得传统媒体的影响力有所下降。广告收入减少、受众流失等问题迫使传统媒体必须进行改革创新。

#### 2.2 传统媒体的应对策略

为了适应新媒体时代的发展，传统媒体采取了一系列应对策略。包括数字化转型，加强与新媒体的融合，以及优化内容生产等。通过这些策略，传统媒体力图在新时代中找到自己的位置，实现可持续发展。

#### 2.3 传统媒体与新媒体的融合发展

融合发展成为了传统媒体在新媒体时代的重要选择。这不仅仅是技术的融合，更是传播理念、内容生产、经营模式等方面的深度融合。通过这种融合，传统媒体可以借鉴新媒体的优势，同时发挥自身在内容制作、品牌影响力等方面的长处，实现优势互补。

### 第三章：新媒体时代监督的发展格局

#### 3.1 新媒体监督的现状

当前，新媒体监督已经成为社会监督的重要组成部分。通过微博、微信等社交平台，公众能够迅速关注并传播社会事件，对公共事务进行监督。新媒体监督在推动公共事件解决、提高政府透明度等方面发挥了积极作

## 提炼目录

In [28]:
content = """
```
重庆市市场监督新媒体监督系统升级改造建议

（一）项目背景与需求分析
一、现有系统问题概述
二、新媒体监督趋势分析
三、升级改造的必要性

（二）升级目标与工作内容
一、系统性能优化
二、新媒体内容监测模块升级
   这部分工作包括加强新媒体内容识别准确性，提高信息处理速度。
三、数据分析与报告模块
   工作职责涉及数据挖掘、趋势分析，建立定期报告机制。
四、用户交互体验优化
   目标是提升用户操作便捷性，增强系统互动性。

（三）技术方案与实施步骤
一、技术选型与平台构建
二、系统架构优化
三、分阶段实施计划
   包括初期测试、中期推广、后期完善等步骤。

（四）风险评估与应对措施
一、技术风险评估
二、数据安全策略
三、应急预案制定

（五）经费预算与资源分配
一、预算概览
二、人力资源配置
三、设备与软件投入

（六）调研与评估
一、市场调研
二、用户需求调研
三、效果评估机制

（七）项目时间表与里程碑
一、项目启动阶段
二、研发与测试阶段
三、上线运行与优化阶段

（八）后期维护与持续改进
一、系统维护计划
二、用户反馈收集与处理
三、长期升级规划
```

以上是根据您提供的示例和需求分析制定的重庆市市场监督新媒体监督系统升级改造建议的主要章节，保持了语言简练，符合指导原则。
"""

In [33]:
translate_prompt_kv = {
  "goal": """
    请根据输入的内容提炼出目录结构，以便后续扩写。
    你必须按照一个层级提炼，并使用JSON格式输出。
    """,

  "instruction": """
    1. 语言尽量简练，不要啰嗦，不要增加补充提问。
    """,

  "demo": """
    ```json
    [
        {
            "名称": 01_xxxx,
            "摘要": xxx
        },
        {
            "名称": 03_xxxx,
            "摘要": xxx
        },
        ...
        
    ]
    ```
    """
}

book_chain = book.get_chain(**translate_prompt_kv)
for chunk in book_chain.stream(content):
    print(chunk.content, end="")

```json
[
    {
        "名称": "01_项目背景与需求分析",
        "摘要": "概述现有系统问题、新媒体监督趋势及升级改造的必要性。"
    },
    {
        "名称": "02_升级目标与工作内容",
        "摘要": "明确系统性能优化、新媒体内容监测、数据分析与报告、用户交互体验等升级内容。"
    },
    {
        "名称": "03_技术方案与实施步骤",
        "摘要": "介绍技术选型、架构优化及分阶段实施计划。"
    },
    {
        "名称": "04_风险评估与应对措施",
        "摘要": "分析技术风险、数据安全策略及应急预案。"
    },
    {
        "名称": "05_经费预算与资源分配",
        "摘要": "提供预算概览、人力资源配置及设备与软件投入。"
    },
    {
        "名称": "06_调研与评估",
        "摘要": "进行市场调研、用户需求调研及效果评估机制设立。"
    },
    {
        "名称": "07_项目时间表与里程碑",
        "摘要": "规划项目启动、研发测试、上线运行及优化阶段。"
    },
    {
        "名称": "08_后期维护与持续改进",
        "摘要": "制定系统维护计划、收集用户反馈及规划长期升级。"
    }
]
```